In [ ]:
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Activation, Flatten, Softmax
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D
import pickle


In [ ]:
##################################
path= 'Notes'
testRatio=0.2
valRatio=0.2
imageDimensions=(32,32,3)

batchSizeVal= 50
epochsVal= 30
stepsPerEpoch = 20

####################################
images =[]
count=0
classNo =[]
List= os.listdir(path)
print("Total no of clases detected",len(List))
noOfClasses = len(List)
print("Importing classes....")
for x in range (0,noOfClasses):
    PicList = os.listdir(path+"/"+str((x)))

    for y in PicList:
         curImg = cv2.imread(path+"/"+str(x)+"/"+y)
         curImg =cv2.resize(curImg,(32,32))
         images.append(curImg)
         classNo.append(count)
    print(count,end= " ")
    count +=1
print(" ")

images = np.array(images)
classNo = np.array(classNo)
print(images.shape)
## Spliting the data
x_train, x_test,y_train,y_test = train_test_split(images,classNo,test_size=0.2)
x_train,x_validation,y_train,y_validation=train_test_split(x_train,y_train,test_size=valRatio)
print(x_train.shape)
print(x_test.shape)
print(x_validation.shape)


numOfSamples= []
for x in range(0,noOfClasses):
    ##print(len(np.where(y_train==0)[0]))
    numOfSamples.append(len(np.where(y_train==x)[0]))
print(numOfSamples)

plt.figure(figsize=(10,5))
plt.bar(range(0,noOfClasses),numOfSamples)
plt.title("No of Images for each Class")
plt.xlabel("Class ID")
plt.ylabel("Number of Images")
plt.show()

def preProcessing(img):
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = cv2.equalizeHist(img)
        img = img/255
        return img

#img = preProcessing(x_train[30])
#img=cv2.resize(img,(300,300))
#cv2.imshow("PreProcessed",img)
#cv2.waitKey(0)

x_train=np.array(list(map(preProcessing,x_train)))
x_test=np.array(list(map(preProcessing,x_test)))
x_validation=np.array(list(map(preProcessing,x_validation)))

x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)
x_validation = x_validation.reshape(x_validation.shape[0],x_validation.shape[1],x_validation.shape[2],1)

dataGen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10)

dataGen.fit(x_train)
y_train=to_categorical(y_train,noOfClasses)
y_test=to_categorical(y_test,noOfClasses)
y_validation=to_categorical(y_validation,noOfClasses)

def myModel():
    noOfFilters= 60
    sizeOfFilter1=(5,5)
    sizeOfFilter2= (3,3)
    sizeOfPool= (2,2)
    noOfNode = 500

    model = Sequential()
    model.add((Conv2D(noOfFilters,sizeOfFilter1,input_shape=(imageDimensions[0],imageDimensions[1],1), activation='relu')))
    model.add((Conv2D(noOfFilters, sizeOfFilter1, activation='relu')))
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation='relu')))
    model.add((Conv2D(noOfFilters//2, sizeOfFilter2, activation='relu')))
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(noOfNode,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(noOfClasses,activation='softmax'))
    model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

model = myModel()
print(model.summary())

history = model.fit(dataGen.flow(x_train,y_train,batch_size=batchSizeVal),steps_per_epoch=stepsPerEpoch,
                    epochs=epochsVal,validation_data=(x_validation,y_validation),
                    shuffle=1)

"""
new_img = cv2.imread()
pred = model.predict(new_img)

"""

# print(history)
plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training','validation'])
plt.title('Loss')
plt.xlabel('epoch')
plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','validation'])
plt.title('Accuracy')
plt.xlabel('epoch')
plt.show()
score = model.evaluate(x_test,y_test,verbose=0)
print('Test Score = ',score[0])
print('Test Accuracy =', score[1])

pickle_out = open("model_trained.pkl","wb")
pickle.dump(model,pickle_out)
pickle_out.close()


In [ ]:

##########################################
width = 640
height = 480
threshold = 0.85
##########################################
cap = cv2.VideoCapture(0)
cap.set(3,width)
cap.set(4,height)

pickle_in = open("model_trained.pkl","rb+")
# model = pickle.loads(pickle_in)

def preProcessing(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img / 255
    return img

#while True:
success, imgOriginal = cap.read()
imgOriginal = cv2.imread("C:\\Users\\Sree\\PycharmProjects\\serious\\Notes\\1\\h1.jpg")
img = np.asarray(imgOriginal)
img = cv2.resize(img,(32,32))
img = preProcessing(img)
cv2.imshow("Processed Image",img)
img = img.reshape(1,32,32,1)

#Predict
#classIndex = int(model.predict_classes(img))
#print(classIndex)
predictions = model.predict(img)[0]
print(predictions)
probVal= np.amax(predictions)
# print(classIndex,probVal)

# if probVal> threshold:
  #     cv2.putText(imgOriginal,str(classIndex) +"  "+str(probVal),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
cv2.imshow("Original Image",imgOriginal)
cv2.waitKey(0)
if cv2.waitKey(1) &0xFF == ord('q'):
   break

cv2.destroyAllWindows()
cap.release()

In [ ]:
cv2.destroyAllWindows()
cap.release()


In [ ]:

# def sliding_window(image, stepSize):
# 	imgarr=[]
# 	# xarr = []
# 	# yarr = []

# 	Eline[]=[[115,][212][320][426]]
# 	Gline[]=[[109][206][314][420]]
# 	#F if condition stating that it touches both.
# 	Cline[]=[[120][216][325][432]]
# 	Dline[]=[[115 &&119][212&&216][320&&324][426&&430]]
# 	Fline[]=[[109 && 115][206&&212][320&&314][426&&420]]
# 	#Aline[]=[[]]
# 	#Bline[]=[[]]
# 	##################################################
# Cl2[]=[[115&&119],[216],[314],[430]]
# cl1[]=[[115],[115 &&119],[109],[109 && 115],[212][320][426][206][314][420][120][216][325][432][212&&216][320&&324][426&&430][206&&212][320&&314][426&&420]]

image = cv2.imread("MUSI.jpeg")

class0=[]
class1=[]
class2=[]
class3=[]
class4=[]


stepSize = 32
threshold = 0.85

# slide a window across the image
for y in range(0, image.shape[0] - stepSize - 1):
	for x in range(0, image.shape[1] - stepSize - 1):
		# yield the current window
		img=image[y:y+stepSize,x:x+stepSize]
		img = np.asarray(image)
		img = cv2.resize(img,(32,32))
		img = preProcessing(img)
		# cv2.imshow("Processed Image",img)
		img = img.reshape(1,32,32,1)
		predictions = model.predict(img)[0]
		# print(predictions)
		probVal= np.amax(predictions)
		for tempp in predictions:
			if tempp == probVal:
				classIndex = tempp

		# Prediction of the class here
		print(classIndex,probVal)
		if probVal> threshold:
			cv2.putText(imgOriginal,str(classIndex) +"  "+str(probVal),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
cv2.imshow("img", imgOriginal)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Append the coords in xarr and yarr


# for 
# 			if probVal in predictions[0]
# 			class0.append(x,y)
# 			if probVal in predictions[1]
# 			class1.append(x,y)
# 			if probVal in predictions[2]
# 			class2.append(x,y)
# 			if probVal in predictions[3]
# 			class3.append(x,y)
# 			if probVal in predictions[4]
# 			class4.append(x,y)
# 		# Return the xarr and yarr
# 			imgarr.append(img)
# 	return imgarr


In [ ]:
testimage=cv2.imread("download.png")
imarr=sliding_window(testimage,stepSize=32)
count=0
for i in imarr:
    img = np.asarray(i)
    img = cv2.resize(img,(32,32))
    img = preProcessing(img)
    cv2.imshow("Processed Image",img)
    img = img.reshape(1,32,32,1)

    #Predict
    # classIndex = int(model.predict_classes(img))
    #print(classIndex)
    predictions = model.predict(img)[0]
    print(predictions)
    probVal= np.amax(predictions)#insert max function
    count+=1
    #identify index of max 

In [ ]:
"""
You'll get the class names and the corresponding y coords of every symbol.
Compare those y coords of the symbol to the y coords of the horizontal lines to get which line the symbol is touching.

You have which symbol (count) and where is the symbol (which key).
"""

In [ ]:
################################################################################################
line_y_coords = [[94,99,105,109,115],[192,196,202,206,212],[299,304,309,315,320],[406,410,416,420,426]]
for a in range(0, 
    if note in line_y_coords 
    if note in Cline 
        cv2.puttext("C ")
    elif note in Dline 
        cv2.puttext("D ")
    elif note in Eline 
        cv2.puttext("E ")    
    elif note in Fline 
        cv2.puttext("F ")
   
    print(max)